In [263]:
#libraies we need
import numpy as np 
import pandas as pd 
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
#reviewing the data
df=pd.read_csv('train.csv')
l1=df.columns

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('SalePrice',axis=1), df['SalePrice'],                 # Features and target
    test_size=0.2,        # 20% for testing
    random_state=42,      # For reproducibility
)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_train.shape


(1168, 80)

In [264]:
#manual classiication of columns based wether the data is numerical,nominal or ordinal data 
numerical=X_train[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
              'LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
              'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch',
              '3SsnPorch','ScreenPorch','PoolArea','MiscVal']]
nominal=X_train[['Street','Alley','LandContour','LotConfig','Neighborhood','Condition1',
           'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',
           'Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir',
           'GarageType','GarageFinish','MiscFeature','SaleType','SaleCondition']]
ordinal=X_train[['MSSubClass','MSZoning','LotShape','Utilities','LandSlope','OverallQual','OverallCond'
           ,'YearBuilt','YearRemodAdd','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
           'HeatingQC','Electrical','KitchenQual','Functional','FireplaceQu','GarageYrBlt','GarageFinish','GarageQual','GarageCond',
           'PavedDrive','PoolQC','Fence','MoSold','YrSold']]

In [265]:
#Functions to check for dominant columns and excecive nans in order to remove as they are not necessary for the model
def get_low_information_columns(df, dominance_threshold=0.95):
    """
    Finds columns where one category dominates (occurs in >= dominance_threshold proportion of rows).
    Parameters:
    - df: pandas DataFrame
    - column_list: list of columns to check
    - dominance_threshold: float (0 to 1), threshold for dominant category
    
    Returns:
    - List of column names with dominant single-category values
    """
    low_info_cols = []

    for col in df.columns:
        top_freq = df[col].value_counts(normalize=True, dropna=False).values[0]
        if top_freq >= dominance_threshold:
            low_info_cols.append(col)

    return low_info_cols

def get_columns_with_excessive_nans(df, threshold=0.5):
    """
    Returns columns from column_list where the proportion of NaN values exceeds the given threshold.
    Parameters:
    - df: pandas DataFrame
    - column_list: list of column names to check
    - threshold: float between 0 and 1 (e.g. 0.5 means drop if >50% NaNs)
    Returns:
    - List of column names to consider dropping
    """
    total_rows = len(df)
    drop_candidates = []
    for col in df.columns:
        nan_ratio = df[col].isna().sum() / total_rows
        if nan_ratio > threshold:
            drop_candidates.append(col)
    return drop_candidates


In [266]:
#selecting columns to drop based on their dominance and nans number

numerical_drop=['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','BedroomAbvGr','GarageCars']

ordinal_drops=get_low_information_columns(ordinal,0.7)
nominal_drops=get_low_information_columns(nominal,0.7)

nan_numerical=get_columns_with_excessive_nans(numerical,0.7)
nan_nominal=get_columns_with_excessive_nans(nominal,0.7)
nan_ordinal=get_columns_with_excessive_nans(ordinal,0.7)

tot_num_drop=list(set(numerical_drop+nan_numerical))
tot_nom_drop=list(set(nominal_drops+nan_nominal))
tot_ord_drop=list(set(ordinal_drops+nan_ordinal))
tot_num_drop
#dropat

numerical_v2=numerical.drop(tot_num_drop,axis=1)
nominal_v2=nominal.drop(tot_nom_drop,axis=1)
ordinal_v2=ordinal.drop(tot_ord_drop,axis=1)
ordinal_v2=ordinal_v2.drop('MoSold',axis=1)
ordinal_v2.shape


(1168, 16)

In [267]:
ordinal_drops

['MSZoning',
 'Utilities',
 'LandSlope',
 'ExterCond',
 'BsmtCond',
 'BsmtFinType2',
 'Electrical',
 'Functional',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence']

In [268]:
#feature engineering
full_bath=['BsmtFullBath','FullBath']
half_bath=['BsmtHalfBath','HalfBath']
tot_area=['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch',
          'PoolArea']
#Adding all th area columns of the house into a single area of the house
numerical_v2['total_area']=numerical_v2[tot_area].sum(axis=1)
numerical_v2.drop(tot_area,axis=1,inplace=True)
#Adding all the full bathrooms and half bathrooms into a single column
numerical_v2['totalbathreoams']=numerical_v2[full_bath].sum(axis=1)+(numerical_v2[half_bath].sum(axis=1)*0.5)
numerical_v2=numerical_v2.drop(full_bath,axis=1)
numerical_v2=numerical_v2.drop(half_bath,axis=1)


significant=['LowQualFinSF','GrLivArea','KitchenAbvGr','TotRmsAbvGrd']#expected signeficant columns from data understanding
numerical_v2.shape

(1168, 8)

In [269]:
# the correlation matrix
corr_matrix = numerical_v2.corr(numeric_only=True)
# Convert it to long format for Plotly
corr_long = corr_matrix.reset_index().melt(id_vars='index')
corr_long.columns = ['Feature1', 'Feature2', 'Correlation']
# Create Plotly heatmap
fig = px.imshow(
    corr_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Correlation Matrix Heatmap',
    aspect='auto'
)

fig.update_layout(
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis_tickangle=-45
)
fig.show()

In [270]:
numerical_v2=numerical_v2.drop('MiscVal',axis=1)
numerical_v2=numerical_v2.drop('GrLivArea',axis=1)
numerical_v2

,LowQualFinSF,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,total_area,totalbathreoams
0,0,1,5,0,11642.0,2.0
1,0,1,7,1,10686.0,2.5
2,0,1,4,0,10928.0,1.0
3,0,1,7,2,10515.0,2.5
4,0,1,6,1,8317.0,2.0
...,...,...,...,...,...,...
1163,0,1,6,1,12485.0,2.0
1164,0,1,7,2,12023.0,3.0
1165,0,1,5,0,10532.0,2.0
1166,0,1,7,1,10491.0,1.5


In [271]:
#nominal encoding 
#nominal encoding 
encoded_nominal=pd.DataFrame()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Fit and transform only the specified columns
encoded_data = ohe.fit_transform(nominal_v2)

# Get the new column names for encoded features
encoded_cols = ohe.get_feature_names_out(nominal_v2.columns)

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=nominal_v2.index)

# Drop original columns and concatenate the encoded ones

encoded_nominal = pd.concat([encoded_nominal, encoded_df], axis=1)
encoded_nominal.shape


(1168, 85)

In [272]:
#ordinal encoding
#small function to show all the unique values of each column (numerical are sorted) in order to make the order manually for the encoder 
'''for col in ordinal_v2.columns:
    uniques = sorted(ordinal_v2[col].dropna().unique())

    cleaned_uniques = []
    for val in uniques:
        if isinstance(val, (np.integer, int)):
            cleaned_uniques.append(int(val))
        elif isinstance(val, (np.floating, float)):
            # Convert float to int if it's a whole number like 1900.0
            cleaned_uniques.append(int(val) if val.is_integer() else float(val))
        else:
            cleaned_uniques.append(val)

    print(f"{col}: {cleaned_uniques}")
'''


orders_list=[[20, 30, 40, 45, 50, 60, 70, 75, 80, 85, 90, 120, 160, 180, 190],
             ['Reg','IR1', 'IR2', 'IR3'],
             [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
             [1, 2, 3, 4, 5, 6, 7, 8, 9],
             [1872, 1875, 1880, 1882, 1885, 1890, 1892, 1893, 1898, 1900, 1904, 1905, 1906, 1908,
              1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917,1918, 1919, 1920, 1921, 1922, 1923, 
              1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1934, 1935, 1936, 1937, 1938,
              1939, 1940,1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 
              1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,1965, 1966, 1967, 1968, 
              1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
              1983, 1984, 1985,1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 
              1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010],
             [1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 
              1964, 1965, 1966, 1967, 1968, 1969, 1970,1971, 1972, 1973, 1974, 1975, 1976, 1977, 
              1978, 1979, 1980, 1981, 1982, 1983,1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 
              1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
              2006, 2007, 2008, 2009, 2010],
             ['Ex', 'Gd', 'TA', 'Fa'],
             ['Ex', 'Gd', 'TA', 'Fa'],
             [ 'Gd','Av', 'Mn', 'No'],
             ['GLQ','ALQ', 'BLQ', 'Rec', 'LwQ',  'Unf'],
             ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
             ['Ex', 'Gd', 'TA', 'Fa'],
             ['Ex', 'Gd', 'TA', 'Fa', 'Po'],
             [1900, 1906, 1908, 1910, 1914, 1915, 1916, 1918, 1920, 1921, 1922, 1923, 1924, 1925,
              1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939,
              1940, 1941, 1942, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955,
              1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
              1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
              1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
              1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010],
              ['Fin', 'RFn', 'Unf'],
               [2006, 2007, 2008, 2009, 2010]]

ordinal_cols=ordinal_v2.columns
encoder = OrdinalEncoder(categories=orders_list,handle_unknown='use_encoded_value', unknown_value=-1)
ordinal_v2[ordinal_cols] = encoder.fit_transform(ordinal_v2[ordinal_cols])
ordinal_encoded=ordinal_v2
ordinal_encoded.shape



(1168, 16)

In [273]:
#final data frame (nesceds scaling and ready for ML)
numerical_v2
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

numerical_v3=scaler.fit_transform(numerical_v2)
numerical_v4=pd.DataFrame(numerical_v3,columns=numerical_v2.columns)
final_train=pd.concat([numerical_v4,encoded_nominal,ordinal_encoded],axis=1)
final_train.shape



(1168, 107)

In [274]:
#encapsulate in a pipeline in the future 

df=pd.read_csv('test.csv')
l1=df.columns
df['1stFlrSF']


0        896
1       1329
2        928
3        926
4       1280
        ... 
1454     546
1455     546
1456    1224
1457     970
1458     996
Name: 1stFlrSF, Length: 1459, dtype: int64

In [275]:
#manual classiication of columns based wether the data is numerical,nominal or ordinal data 
numerical_test=X_test[['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
              'LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
              'TotRmsAbvGrd','Fireplaces','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch',
              '3SsnPorch','ScreenPorch','PoolArea','MiscVal']]
nominal_test=X_test[['Street','Alley','LandContour','LotConfig','Neighborhood','Condition1',
           'Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',
           'Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir',
           'GarageType','GarageFinish','MiscFeature','SaleType','SaleCondition']]
ordinal_test=X_test[['MSSubClass','MSZoning','LotShape','Utilities','LandSlope','OverallQual','OverallCond'
           ,'YearBuilt','YearRemodAdd','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
           'HeatingQC','Electrical','KitchenQual','Functional','FireplaceQu','GarageYrBlt','GarageFinish','GarageQual','GarageCond',
           'PavedDrive','PoolQC','Fence','MoSold','YrSold']]

'''ordinal_v2.shape


ordinal_test=ordinal_test.drop(tot_ord_drop,axis=1)

ordinal_test.drop('MoSold',axis=1,inplace=True)
ordinal_v2.shape
ordinal_test.shape
numerical_test=numerical_test.drop(tot_num_drop,axis=1)
numerical_test.shape
numerical_v2
numerical_test.head()'''

numerical_test=numerical_test.drop(tot_num_drop,axis=1)
nominal_test=nominal_test.drop(tot_nom_drop,axis=1)
ordinal_test=ordinal_test.drop(tot_ord_drop,axis=1)
ordinal_test=ordinal_test.drop('MoSold',axis=1)

#feature engineering

#Adding all th area columns of the house into a single area of the house
numerical_test['total_area']=numerical_test[tot_area].sum(axis=1)
numerical_test.drop(tot_area,axis=1,inplace=True)
#Adding all the full bathrooms and half bathrooms into a single column
numerical_test['totalbathreoams']=numerical_test[full_bath].sum(axis=1)+(numerical_test[half_bath].sum(axis=1)*0.5)
numerical_test=numerical_test.drop(full_bath,axis=1)
numerical_test=numerical_test.drop(half_bath,axis=1)
numerical_test=numerical_test.drop('MiscVal',axis=1)
numerical_test=numerical_test.drop('GrLivArea',axis=1)

#nominal encoding 
encoded_nominal_test=pd.DataFrame()
 

# Fit and transform only the specified columns
encoded_data = ohe.transform(nominal_test)

# Get the new column names for encoded features
encoded_cols = ohe.get_feature_names_out(nominal_test.columns)

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_data, columns=encoded_cols, index=nominal_test.index)

# Drop original columns and concatenate the encoded ones

encoded_nominal_test = pd.concat([encoded_nominal_test, encoded_df], axis=1)


ordinal_cols=ordinal_test.columns
ordinal_test[ordinal_cols] = encoder.transform(ordinal_test[ordinal_cols])
ordinal_encoded_test=ordinal_test
ordinal_encoded_test


ordinal_encoded_test.shape


numerical_test
from sklearn.preprocessing import StandardScaler

numerical_test_v3=scaler.fit_transform(numerical_test)
numerical_test_v4=pd.DataFrame(numerical_test_v3,columns=numerical_test.columns)
final_test=pd.concat([numerical_test_v4,encoded_nominal_test,ordinal_encoded_test],axis=1)
y_train.shape



X_train=final_train
X_test=final_test



In [276]:
#First try using sbase reggresors with average majority voting ensemble
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np

# Define base regressors
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
svr = SVR()

# Train them
lr.fit(final_train, y_train)
rf.fit(final_train, y_train)
svr.fit(final_train, y_train)

# Predict
lr_pred = lr.predict(final_test)
rf_pred = rf.predict(final_test)
svr_pred = svr.predict(final_test)

# Simple average
avg_pred = (lr_pred + rf_pred + svr_pred) / 3

# Evaluate
mse = mean_squared_error(y_test, avg_pred)
r2 = r2_score(y_test, avg_pred)

print(f"Simple Averaging MSE: {mse:.2f}")
print(f"Simple Averaging R² Score: {r2:.4f}")

Simple Averaging MSE: 1849286806.59
Simple Averaging R² Score: 0.7589


In [277]:
#more powerful reggresors (xg boost,LGBMRegressor)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Initialize models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42, verbosity=0)
lgb = LGBMRegressor(random_state=42)

# Fit models
lr.fit(final_train, y_train)
rf.fit(final_train, y_train)
xgb.fit(final_train, y_train)
lgb.fit(final_train, y_train)

# Predict on test set
lr_pred = lr.predict(final_test)
rf_pred = rf.predict(final_test)
xgb_pred = xgb.predict(final_test)
lgb_pred = lgb.predict(final_test)

# Average predictions
avg_pred = (lr_pred + rf_pred + xgb_pred + lgb_pred) / 4

# Evaluation
mse = mean_squared_error(y_test, avg_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, avg_pred)

print(f"Simple Averaging MSE: {mse:.2f}")
print(f"Simple Averaging RMSE: {rmse:.2f}")
print(f"Simple Averaging R² Score: {r2:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 725
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 76
[LightGBM] [Info] Start training from score 181441.541952
Simple Averaging MSE: 1053286220.63
Simple Averaging RMSE: 32454.37
Simple Averaging R² Score: 0.8627


In [278]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import r2_score

# Example: you already have these predictions from your trained models on X_test
# lr_pred = lr.predict(X_test_encoded)
# rf_pred = rf.predict(X_test_encoded)
# xgb_pred = xgb.predict(X_test_encoded)
# lgb_pred = lgb.predict(X_test_encoded)

# For demonstration, I will assume lr_pred, rf_pred, xgb_pred, lgb_pred are numpy arrays
# y_test is the true target values

def objective(weights):
    # Weighted sum of predictions
    weighted_pred = (
        weights[0] * lr_pred +
        weights[1] * rf_pred +
        weights[2] * xgb_pred +
        weights[3] * lgb_pred
    )
    # We want to maximize R2, so minimize negative R2
    return -r2_score(y_test, weighted_pred)

# Constraint: weights must sum to 1
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

# Bounds: each weight between 0 and 1
bounds = [(0,1)] * 4

# Initial guess: equal weights
init_guess = [0.25, 0.25, 0.25, 0.25]

result = minimize(objective, init_guess, bounds=bounds, constraints=constraints)

best_weights = result.x
best_r2 = -result.fun

print(f"Optimized weights: {best_weights}")
print(f"Optimized R² score: {best_r2:.4f}")

# Calculate final weighted prediction with optimized weights
final_pred = (
    best_weights[0] * lr_pred +
    best_weights[1] * rf_pred +
    best_weights[2] * xgb_pred +
    best_weights[3] * lgb_pred
)

# Optionally calculate MSE and RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, final_pred)
rmse = np.sqrt(mse)

print(f"Weighted Averaging MSE: {mse:.2f}")
print(f"Weighted Averaging RMSE: {rmse:.2f}")


Optimized weights: [0.38394124 0.56662295 0.         0.04943581]
Optimized R² score: 0.8681
Weighted Averaging MSE: 1012033621.60
Weighted Averaging RMSE: 31812.48


In [279]:
#bagging regressors
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Initialize base model (Decision Tree)
base_model = DecisionTreeRegressor(random_state=42)

# Initialize Bagging Regressor with base model
bagging = BaggingRegressor(estimator=base_model,
                           n_estimators=50,
                           random_state=42)

# Fit on training data
bagging.fit(final_train, y_train)

# Predict on test set
y_pred = bagging.predict(final_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Bagging Regressor MSE: {mse:.2f}")
print(f"Bagging Regressor RMSE: {rmse:.2f}")
print(f"Bagging Regressor R² Score: {r2:.4f}")


Bagging Regressor MSE: 1184877105.84
Bagging Regressor RMSE: 34422.04
Bagging Regressor R² Score: 0.8455


In [280]:
#voting reggresor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Initialize individual models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
lgb = LGBMRegressor(random_state=42)

# Initialize Voting Regressor
voting_reg = VotingRegressor(estimators=[
    ('lr', lr),
    ('rf', rf),
    ('xgb', xgb),
    ('lgb', lgb)
])

# Fit Voting Regressor on training data
voting_reg.fit(final_train, y_train)

# Predict on test data
y_pred = voting_reg.predict(final_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Voting Regressor MSE: {mse:.2f}")
print(f"Voting Regressor RMSE: {rmse:.2f}")
print(f"Voting Regressor R² Score: {r2:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 725
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 76
[LightGBM] [Info] Start training from score 181441.541952
Voting Regressor MSE: 1053286220.63
Voting Regressor RMSE: 32454.37
Voting Regressor R² Score: 0.8627


In [281]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Base models
estimators = [
    ('rf', RandomForestRegressor(random_state=42)),
    ('xgb', XGBRegressor(random_state=42)),
    ('lgb', LGBMRegressor(random_state=42))
]

# Meta-model: Ridge Regression (simple, effective)
meta_regressor = Ridge(alpha=1.0)

# Initialize Stacking with cv=10
stacking_reg = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_regressor,
    cv=10,
    n_jobs=-1
)

# Train
stacking_reg.fit(final_train, y_train)

# Predict
y_pred = stacking_reg.predict(final_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Stacking Regressor with cv=10 MSE: {mse:.2f}")
print(f"Stacking Regressor with cv=10 RMSE: {rmse:.2f}")
print(f"Stacking Regressor with cv=10 R² Score: {r2:.4f}")


Stacking Regressor with cv=10 MSE: 1184466318.14
Stacking Regressor with cv=10 RMSE: 34416.08
Stacking Regressor with cv=10 R² Score: 0.8456


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Step 1: Split your existing training data into base training and holdout (for blending)
X_base_train, X_holdout, y_base_train, y_holdout = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

# Step 2: Initialize base models
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
lgb = LGBMRegressor(random_state=42)

# Step 3: Train base models on the base training set only
rf.fit(X_base_train, y_base_train)
xgb.fit(X_base_train, y_base_train)
lgb.fit(X_base_train, y_base_train)

# Step 4: Generate predictions of base models on the holdout set (meta-model training data)
rf_holdout_pred = rf.predict(X_holdout)
xgb_holdout_pred = xgb.predict(X_holdout)
lgb_holdout_pred = lgb.predict(X_holdout)

# Step 5: Stack these predictions as features to train the meta-model (blender)
X_blend_train = np.column_stack((rf_holdout_pred, xgb_holdout_pred, lgb_holdout_pred))

# Step 6: Initialize and train meta-model (e.g., Ridge)
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_blend_train, y_holdout)

# Step 7: Predict base models on the original test set
rf_test_pred = rf.predict(X_test)
xgb_test_pred = xgb.predict(X_test)
lgb_test_pred = lgb.predict(X_test)

# Step 8: Stack test predictions as input for meta-model
X_blend_test = np.column_stack((rf_test_pred, xgb_test_pred, lgb_test_pred))

# Step 9: Make final blended predictions
y_pred = meta_model.predict(X_blend_test)

# Step 10: Evaluate the blended predictions
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Blending MSE: {mse:.2f}")
print(f"Blending RMSE: {rmse:.2f}")
print(f"Blending R² Score: {r2:.4f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 697
[LightGBM] [Info] Number of data points in the train set: 934, number of used features: 70
[LightGBM] [Info] Start training from score 181034.540685
Blending MSE: 1271865026.99
Blending RMSE: 35663.22
Blending R² Score: 0.8342


In [286]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Re-split to ensure clean training/validation split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1200),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 2),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 2),
        "random_state": 42,
    }

    model = XGBRegressor(**params)
    model.fit(X_tr, y_tr)  # removed early stopping
    preds = model.predict(X_val)
    return r2_score(y_val, preds)

# Optimize
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best hyperparameters:", study.best_trial.params)


[I 2025-05-16 21:06:26,311] A new study created in memory with name: no-name-931572d3-f38a-4d1a-9c22-c743bed27893
[I 2025-05-16 21:06:28,702] Trial 0 finished with value: 0.8483250141143799 and parameters: {'n_estimators': 766, 'max_depth': 8, 'learning_rate': 0.157678805850236, 'subsample': 0.962307875928265, 'colsample_bytree': 0.6493699114125192, 'gamma': 4.595263436350335, 'reg_alpha': 0.6784589423513276, 'reg_lambda': 1.611138017081778}. Best is trial 0 with value: 0.8483250141143799.
[I 2025-05-16 21:06:29,939] Trial 1 finished with value: 0.8424611687660217 and parameters: {'n_estimators': 595, 'max_depth': 8, 'learning_rate': 0.2823724672505149, 'subsample': 0.6336711175981, 'colsample_bytree': 0.81966782536045, 'gamma': 1.0963715038886424, 'reg_alpha': 1.5910966367000514, 'reg_lambda': 1.2891854761641113}. Best is trial 0 with value: 0.8483250141143799.
[I 2025-05-16 21:06:33,291] Trial 2 finished with value: 0.8662516474723816 and parameters: {'n_estimators': 988, 'max_depth'

Best hyperparameters: {'n_estimators': 686, 'max_depth': 3, 'learning_rate': 0.023961403365515, 'subsample': 0.7869792280350976, 'colsample_bytree': 0.6006680762021221, 'gamma': 2.4406762215636855, 'reg_alpha': 0.9433811696912263, 'reg_lambda': 1.523233650171723}


In [287]:
from xgboost import XGBRegressor

# Use best parameters from Optuna study
best_params = study.best_trial.params

# You may want to re-specify fixed params like random_state
best_params["random_state"] = 42

# Train final model on full training set
final_xgb_model = XGBRegressor(**best_params)
final_xgb_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = final_xgb_model.predict(X_test)

from sklearn.metrics import r2_score, mean_squared_error
print("Optimized R² Score:", r2_score(y_test, y_pred))
print("Optimized MSE:", mean_squared_error(y_test, y_pred))


Optimized R² Score: 0.8709053993225098
Optimized MSE: 990197568.0


In [289]:
#catboost
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Example data (replace this with your actual dataset)


# Split the data
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Optuna objective
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "depth": trial.suggest_int("depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_strength": trial.suggest_float("random_strength", 1e-9, 10),
        "loss_function": "RMSE",
        "verbose": 0,
        "random_state": 42
    }

    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=30, verbose=0)
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds)

# Run optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, timeout=600)

# Train final model
best_params = study.best_params
final_model = CatBoostRegressor(**best_params, loss_function="RMSE", verbose=0, random_state=42)
final_model.fit(X_train_val, y_train_val)

# Evaluate
y_pred = final_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Best Parameters:", best_params)
print("MSE:", mse)
print("R² Score:", r2)


[I 2025-05-16 21:15:42,471] A new study created in memory with name: no-name-c4942386-8fb7-4fc6-b97b-73ed125ea9ab


[I 2025-05-16 21:15:44,672] Trial 0 finished with value: 0.2226555790691261 and parameters: {'iterations': 830, 'depth': 5, 'learning_rate': 0.0966466423931404, 'l2_leaf_reg': 4.304040520108539, 'bagging_temperature': 0.9425233024067794, 'border_count': 86, 'random_strength': 7.001578246825316}. Best is trial 0 with value: 0.2226555790691261.
[I 2025-05-16 21:15:49,504] Trial 1 finished with value: 0.23134822751090198 and parameters: {'iterations': 355, 'depth': 10, 'learning_rate': 0.1430307018057238, 'l2_leaf_reg': 4.746955337374125, 'bagging_temperature': 0.7646609368084064, 'border_count': 98, 'random_strength': 8.724165047454987}. Best is trial 0 with value: 0.2226555790691261.
[I 2025-05-16 21:15:53,753] Trial 2 finished with value: 0.22769807821470966 and parameters: {'iterations': 999, 'depth': 9, 'learning_rate': 0.09126692647629417, 'l2_leaf_reg': 6.817546843483595, 'bagging_temperature': 0.7657586687650014, 'border_count': 52, 'random_strength': 1.0797700374652552}. Best is 

Best Parameters: {'iterations': 907, 'depth': 7, 'learning_rate': 0.09094272132622196, 'l2_leaf_reg': 5.029918349115874, 'bagging_temperature': 0.6941746125476821, 'border_count': 203, 'random_strength': 7.709115572461209}
MSE: 0.19565940704194254
R² Score: 0.8506882887015954
